# Setting up toffy

There are three parts to this notebook.
1. The first part creates the necessary folders that toffy is expecting, and **only needs to be run the first time you install it on a new CAC**. 
2. The second part updates the co-registration parameters between the slide image (optical image) and the stage coordinates. This **needs to be run when Ionpath changes the co-registration, which usually happens after a column realignment**
3. The third part generates a tuning curve to correct for shifts in instrument sensitivity, and **only needs to be run once per instrument**

In [ ]:
# imports
import sys
sys.path.append('../')

from datetime import datetime
import json
import numpy as np
import os
from sklearn.linear_model import LinearRegression

from toffy import tiling_utils, normalize
from ark.utils import io_utils

## 1. Create necessary folders

**If this is the first time toffy has been installed on your CAC, run the following cell to generate the folders necessary.**


More information on the uses and locations of the directories in toffy can be found in the [README](https://github.com/angelolab/toffy#directory-structure).

In [ ]:
folders = ['D:\\Extracted_Images', 'D:\\Rosetta_Compensated_Images', 'D:\\Normalized_Images', 'D:\\Cohorts',
           'C:\\Users\\Customer.ION\\Documents\\run_metrics', 'C:\\Users\\Customer.ION\\Documents\\watcher_logs',
           'C:\\Users\\Customer.ION\\Documents\\tiled_run_jsons', 
           'C:\\Users\\Customer.ION\\Documents\\autolabeled_tma_jsons', 
           'C:\\Users\\Customer.ION\\Documents\\panel_files', 'C:\\Users\\Customer.ION\\Documents\\normalization_curve', 
           'C:\\Users\\Customer.ION\\Documents\\rosetta_testing',
           'C:\\Users\\Customer.ION\\Documents\\rosetta_matrices']

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

## 2. Update coregistration parameters
In order to display the location of FOVs on the slide image, we need to map the location of pixels in the image to stage coordinates on the sldie. This notebook allows you to generate this information, which can change over time for a number of reasons. We use the ficudial marks, the dots on each side of the slide, to accomplish this. 

### Set stage and optical coordinates of the fiducials

The code below will automatically prompt you to specify the coordinates of the top 3 fiducial pairs in both stage coordinate and corresponding optical coordinate values.

For each fiducial:

1: Use the pop-out window to open the slide image on the commercial instrument viewer.

2: Open the developer console. Right click anywhere on the slide image, this menu should open:

<div>
    <img src="img/inspect_menu.png" width="150" height="137"/>
</div>


Click `Inspect`. The developer console should appear on the right side:

<div>
    <img src="img/developer_console.png" width="254" height="240"/>
</div>

3: In the console (above the tabs `What's New` and `Console`), type `logger.level=4` and hit `Enter`

<div>
    <img src="img/logger_level.png" width="253" height="76"/>
</div>


4: Click on one of the slide image fiducials. Retrieve the stage coordinates on top of the image next to `Target Point`:

<div>
    <img src="img/stage_coords.png" width="164" height="21"/>
</div>



Retrieve the corresponding optical coordinates from the console:

<div>
    <img src="img/optical_coords.png" width="254" height="51"/>
</div>



5: For each fiducial, we prompt you for the stage x-coordinate, stage y-coordinate, optical x-coordinate, and optical y-coordinate values separately. Enter the correct value for the coordinate and hit `Enter` to continue.

6: The co-registration parameter settings will be saved to the `toffy` folder.

In [ ]:
fiducial_info = tiling_utils.read_fiducial_info()

### Generate new co-registration parameters

In [ ]:
coreg_params = tiling_utils.generate_coreg_params(fiducial_info)

### Save co-registration settings

In [ ]:
tiling_utils.save_coreg_params(coreg_params)

## 3. Generate sensitivity tuning curve
The sensitivity of the detector will slowly decay over time, decreasing the counts recorded for the same amount of true signal. The Median Pulse Height (MPH) can be used as metric to evaluate the sensitivity of the detector.

We create a tuning curve to relate MPH to the percentage of max signal to understand their relationship.

Your curve should look like the image below. It's okay if your values are a bit different, but the shape of the curve should be qualitatively the same. The curve will be saved in the `sweep_path` folder you define below.
<figure>
    <img src="img/tuning_curve.png" width="500" style="display=block; margin:auto"/>
    <figcaption> The x axis is the Detector Gain, y axis is the % of Max Signal </figcaption>
</figure>

### Identify detector sweep
The first step is selecting a detector sweep. The goal is for this sweep to cover the range of values most often seen during image acqusition. Therefore, it's best to pick a sweep where the suggested change in voltage following the sweep was less than 50V.

In [ ]:
# pick a name for the sweep, such as the date it was run
sweep_name = '20220417_pmma'

In [ ]:
# create a new folder with the sweep name
normalization_dir = 'C:\\Users\\Customer.ION\\Documents\\normalization_curve'
sweep_path = os.path.join(normalization_dir, sweep_name)
os.makedirs(sweep_path)

Now, copy all of the FOVs from the sweep into the newly created folder, which can be found in *C:\\Users\\Customer.ION\\Documents\\normalization_curve*

### Create tuning curve
We'll then use these FOVs in order to create the curve. Sometimes extreme counts can interfere with the curve generation; you can adjust the default range below if errors occur.

In [ ]:
# define masses to use
normalize.create_tuning_function(sweep_path=sweep_path, count_range=(0, 3000000))